In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme('document')


# Getting Started 入门指南


Welcome to our introductory notebook on GemPy! Here, we will cover the essentials of GemPy, introducing you to the core concepts of
geomodeling and demonstrating how you can leverage these to create your own geological models. We will guide you through building a
model from scratch, based on a conceptual 2D cross-section with boreholes. This simple example will highlight key workflow steps 
and structural features that GemPy can model.

欢迎阅读我们的 GemPy 入门笔记本！在这里，我们将介绍 GemPy 的基本要素，向您介绍地质建模的核心概念，并演示如何利用这些概念创建您自己的地质模型。我们将指导您根据带有钻孔的概念性 2D 横截面从头开始构建模型。这个简单的示例将突出显示 GemPy 可以建模的关键工作流程步骤和结构特征。

## Installation 安装

https://docs.gempy.org/installation.html

## Setting Up Our Environment: Importing Libraries 设置我们的环境：导入库

To work with Python packages in our notebook, we need to import them first. Let's start with that. In the following cell, we will
import GemPy and its GemPy viewer module, which we will be using extensively. Additionally, we will use NumPy for various functions,
as well as Matplotlib and some of its specific modules/functions for visualizing our data and results in 2D and 3D.

要在我们的笔记本中使用 Python 包，我们需要先导入它们。让我们从这里开始。在下面的单元格中，我们将导入 GemPy 及其 GemPy 查看器模块，我们将广泛使用它们。此外，我们将使用 NumPy 执行各种功能，以及 Matplotlib 及其一些特定模块/函数，以便在 2D 和 3D 中可视化我们的数据和结果。

Set environmental variable DEFAULT_BACKEND = PYTORCH

设置环境变量 DEFAULT_BACKEND = PYTORCH

In [ ]:
import os

os.environ["DEFAULT_BACKEND"] = "PYTORCH"

# Importing GemPy and viewer
import gempy_viewer as gpv
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Auxiliary libraries
import numpy as np

import gempy as gp

# sphinx_gallery_thumbnail_number = 11

Main Classes and Objects in GemPy GemPy 中的主要类和对象

""""""""""""""""""""""""""""""""" 

GemPy uses Python classes and objects to store and manipulate the data used for modeling. This object-oriented approach helps make
the code more modular, reusable, and easier to understand. Each class represents a different aspect of the geological modeling process,
and objects are instances of these classes that contain specific data and methods to operate on that data.

GemPy 使用 Python 类和对象来存储和操作用于建模的数据。这种面向对象的方法有助于使代码更加模块化、可重用且更易于理解。每个类代表地质建模过程的不同方面，对象是这些类的实例，包含特定数据和对该数据进行操作的方法。

Here are the main data classes:

以下是主要的数据类：

-  :obj:`gempy.core.data.GeoModel`
-  :obj:`gempy.core.data.StructuralFrame`
-  :obj:`gempy.core.data.StructuralGroup`
-  :obj:`gempy.core.data.StructuralElement`
-  :obj:`gempy.core.data.SurfacePointsTable`
-  :obj:`gempy.core.data.OrientationsTable`
-  :obj:`gempy.core.data.Grid`

## Setting Up a Model: Initializing Objects and Input Data 设置模型：初始化对象和输入数据

Before computing a geological model, we need to set up the relevant objects and input data on which the model will be based. Data can 
be input in various ways. One way is to start from scratch and manually input surface points and orientations. Another way is to import 
existing data from a file, such as a CSV. Here, we will start with the first method to showcase the essential elements and rules needed 
to build a model in GemPy.

在计算地质模型之前，我们需要设置模型所基于的相关对象和输入数据。数据可以通过多种方式输入。一种方法是从头开始手动输入表面点和方向。另一种方法是从文件（例如 CSV）导入现有数据。在这里，我们将从第一种方法开始，展示在 GemPy 中构建模型所需的基本要素和规则。

## Starting a Model from Scratch Based on a Cross-Section 基于横截面从头开始构建模型

This first example will be based on a conceptual cross-section that includes data from three boreholes in a line. Let's start by loading 
the image of this cross-section and plotting it using Matplotlib. For this example, we assume that the size of the image corresponds to 
the real extent of the data. We will see that extent in the plot and also return it by looking at the shape of the image file:

第一个示例将基于一个概念性的横截面，其中包括来自一条线上三个钻孔的数据。让我们首先加载此横截面的图像并使用 Matplotlib 绘制它。对于此示例，我们假设图像的大小对应于数据的实际范围。我们将在图中看到该范围，并通过查看图像文件的形状来返回它：

In [ ]:
img = mpimg.imread('boreholes_concept.png')
plt.imshow(img, origin='upper', alpha=.8)
img.shape[:2]

OK, so we will base our model creation on this cross-section. Let's get started. To initialize our model, we create a `gempy.Model` object. 
This object will contain all other data structures and necessary functionality. Here’s what we will do:

1. **Name the Model**: Assign a name to our model.

2. **Define Extent**: Specify the extent in x, y, and z. The extent should make sense depending on your use case and should enclose all 
   relevant data in a representative space. For this example, we align the extent with the cross-section we imported:

    - **X** is parallel to the section.
    - **Y** is perpendicular. Since we have no data along y, a narrow extent makes sense. We choose an extent of 400, defining it as -200 to 200, 
      placing the cross-section at y=0 (in the middle).
    - **Z** representing depth, takes a negative value since we are modeling the subsurface.

3. **Initialize Structural Framework**: Set up a default structural framework.

4. **Define either resolution or refinement**: In GemPy 3, you can use either regular grids or octrees.
    - **Regular grids**: Define a resolution (and refinement=None). A medium resolution of 50x50x50, for example, results in 125,000 voxels.
      Model voxels are prisms, not cubes, so resolution can differ from extent. Avoid exceeding 100 cells in each direction (1,000,000 voxels)
      to prevent high computational costs.
    - **Octrees**: Define a level of refinement (and resolution=None). Higher refinement levels increase computational costs.

好的，我们将基于此横截面创建模型。让我们开始吧。为了初始化我们的模型，我们创建一个 `gempy.Model` 对象。此对象将包含所有其他数据结构和必要的功能。以下是我们将要做的事情：

1. **命名模型**：为我们的模型指定一个名称。

2. **定义范围**：指定 x、y 和 z 的范围。范围应根据您的用例而定，并应在代表性空间中包含所有相关数据。对于此示例，我们将范围与我们导入的横截面对齐：

    - **X** 平行于剖面。
    - **Y** 垂直。由于我们在 y 方向上没有数据，因此较窄的范围是有意义的。我们选择 400 的范围，将其定义为 -200 到 200，将横截面放置在 y=0（中间）。
    - **Z** 代表深度，取负值，因为我们正在对地下进行建模。

3. **初始化结构框架**：设置默认的结构框架。

4. **定义分辨率或细化**：在 GemPy 3 中，您可以使用规则网格或八叉树。
    - **规则网格**：定义分辨率（且 refinement=None）。例如，50x50x50 的中等分辨率会产生 125,000 个体素。模型体素是棱柱体，而不是立方体，因此分辨率可以与范围不同。避免在每个方向上超过 100 个单元格（1,000,000 个体素），以防止计算成本过高。
    - **八叉树**：定义细化级别（且 resolution=None）。较高的细化级别会增加计算成本。

.. admonition:: Note on choice of modeling grids 关于建模网格选择的说明

   Which type of grid is used depends on the use case. Note that as of the current version of GemPy 3, 
   the rendering of surfaces uses dual-contouring, which is based on octrees. So even if you choose regular grids, octree-based computing will
   be executed additionally in order to render the surfaces in 3D.

   使用哪种类型的网格取决于用例。请注意，截至目前的 GemPy 3 版本，表面的渲染使用基于八叉树的双等值线（dual-contouring）。因此，即使您选择规则网格，也会额外执行基于八叉树的计算以在 3D 中渲染表面。

In [ ]:
geo_model = gp.create_geomodel(
    project_name='Model1',
    extent=[0, 780, -200, 200, -582, 0],
    resolution=(50, 50, 50),
    # refinement=4, # We will use octrees
    structural_frame=gp.data.StructuralFrame.initialize_default_structure()
)

## The `geo_model` Object `geo_model`对象

The :obj:`gempy.core.data.GeoModel` object we just initialized contains all the essential information for constructing our model, 
such as the parameters defined above,
and the input data we will introduce further below. You can access different types of information by accessing the attributes. For instance, you
can retrieve the coordinates of our modeling grid as follows:

我们刚刚初始化的 :obj:`gempy.core.data.GeoModel` 对象包含构建模型所需的所有基本信息，例如上面定义的参数，以及我们将在下面进一步介绍的输入数据。您可以通过访问属性来检索不同类型的信息。例如，您可以按如下方式检索建模网格的坐标：

In [ ]:
geo_model.grid

> Out:
Grid(values=array([[   7.8 , -196.  , -576.18],
       [   7.8 , -196.  , -564.54],
       [   7.8 , -196.  , -552.9 ],
       ...,
       [ 772.2 ,  196.  ,  -29.1 ],
       [ 772.2 ,  196.  ,  -17.46],
       [ 772.2 ,  196.  ,   -5.82]], shape=(125000, 3)), length=array([], dtype=float64), _octree_grid=None, _dense_grid=RegularGrid(resolution=array([50, 50, 50]), extent=array([   0.,  780., -200.,  200., -582.,    0.]), values=array([[   7.8 , -196.  , -576.18],
       [   7.8 , -196.  , -564.54],
       [   7.8 , -196.  , -552.9 ],
       ...,
       [ 772.2 ,  196.  ,  -29.1 ],
       [ 772.2 ,  196.  ,  -17.46],
       [ 772.2 ,  196.  ,   -5.82]], shape=(125000, 3)), mask_topo=array([], shape=(0, 3), dtype=bool), _transform=None), _custom_grid=None, _topography=None, _sections=None, _centered_grid=None, _transform=None, _octree_levels=-1)

The `geo_model` object also contains the structural frame of our model, i.e., information about our main structural groups (also referred to as
series or stacks in our model), and their sequential and geological relationships with one another. Each group can contain several elements,
which can be surfaces representing the **bottom** interfaces of lithological units or fault planes. Each structural group also has a relation
type that defines the relation of the structural elements to others. The relation type is 'erode' by default but can also be 'onlap' or 'fault' 
(more about this later). The structural frame also contains information about fault relationships, i.e., which elements are affected by which 
faults. Let's look at our default structural frame:

`geo_model` 对象还包含我们模型的结构框架，即关于我们的主要结构组（在我们的模型中也称为系列或堆栈）的信息，以及它们彼此之间的顺序和地质关系。每个组可以包含多个元素，这些元素可以是代表岩性单元**底部**界面的表面或断层平面。每个结构组也有一个关系类型，定义了结构元素与其他元素的关系。关系类型默认为 'erode'（侵蚀），但也可以是 'onlap'（超覆）或 'fault'（断层）（稍后会详细介绍）。结构框架还包含有关断层关系的信息，即哪些元素受哪些断层影响。让我们看看我们的默认结构框架：

In [ ]:
geo_model.structural_frame

As you can see, by default, there is one element called 'surface 1' and no faults. However, by default, GemPy actually not only starts off 
with this 'surface 1' but also a 'basement' unit which is always present. We can see this using the following function:

如您所见，默认情况下，有一个名为 'surface 1' 的元素，没有断层。然而，默认情况下，GemPy 实际上不仅从这个 'surface 1' 开始，而且还有一个始终存在的 'basement'（基底）单元。我们可以使用以下函数看到这一点：

In [ ]:
geo_model.structural_frame.structural_elements

> out:
[Element(
    name=surface1,
    color=#015482,
    is_active=True
), Element(
    name=basement,
    color=#9f0052,
    is_active=True
)]

We can also rename our structural elements and assign colors as needed. This will later become relevant for the legend in our plots of the
data and generated model. Let's assume we already know our uppermost surface, 'surface 1,' is a limestone unit. Let's also ensure it is
represented by the same color as displayed in the cross-section. For this, we input hex color codes.

我们还可以根据需要重命名结构元素并分配颜色。这稍后将与我们数据和生成模型的绘图图例相关。让我们假设我们已经知道最上面的表面 'surface 1' 是一个石灰岩单元。让我们还确保它由横截面中显示的相同颜色表示。为此，我们输入十六进制颜色代码。

In [ ]:
geo_model.structural_frame.structural_elements[0].color = '#33ABFF'  # Set 'surface 1' color to blue.
geo_model.structural_frame.structural_elements[1].color = '#570987'  # Set basement color to purple.

geo_model.structural_frame.structural_elements[0].name = 'Limestone'  # Renaming 'surface 1' to 'Limestone'.
geo_model.structural_frame.structural_elements[0]

In [ ]:
geo_model.structural_frame.structural_groups[0].name = 'Deposit. Series'

## Manually Inputting Data 手动输入数据

Now that the `geo_model` object has been created and our first structural element renamed, we can start inputting data by reading it from
the cross-section. We start with location points that represent the boundaries between two lithological units. Let's look at the cross-section 
again. This time we use a `gempy_viewer` function to create a plot that can include the cross-section image, as well as data in the object 
`geo_model` - which is for now empty but we will start adding data next. Let's also add a grid to better read the location of points.

The image cross-section indicates that there are three lithological boundaries, with one dot for each boundary shown in the first two boreholes.
The third borehole doesn't go as deep and only provides one dot, i.e., information about the first boundary. These dots represent the location
of a boundary surface in depth. Remember also that in GemPy, each surface represents the bottom of the unit it is assigned to.

既然已经创建了 `geo_model` 对象并重命名了我们的第一个结构元素，我们可以通过从横截面读取数据来开始输入数据。我们从代表两个岩性单元之间边界的位置点开始。让我们再次查看横截面。这次我们使用 `gempy_viewer` 函数创建一个图，其中可以包含横截面图像，以及对象 `geo_model` 中的数据 - 目前它是空的，但我们接下来将开始添加数据。让我们还添加一个网格以便更好地读取点的位置。

图像横截面表明有三个岩性边界，前两个钻孔中每个边界显示一个点。第三个钻孔没有那么深，只提供了一个点，即关于第一个边界的信息。这些点代表深度中边界表面的位置。还要记住，在 GemPy 中，每个表面代表分配给它的单元的底部。

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

Looking at the plot, we can read the location of the surface points and start adding them to our `geo_model` object. For now, let's focus on 
only the uppermost layer, the limestone, with its bottom boundary represented by the blue dots. Using the function `add_surface_points`, 
we can start adding the positional points to our `geo_model` object. Let's start with only one. From looking at the plot, the first blue
point seems to be located at approximately 250 in the x direction and around 95 meters in depth. Since we assume the section to be at y=0,
we can leave y as 0. We can input that as follows:

查看图表，我们可以读取表面点的位置并开始将它们添加到我们的 `geo_model` 对象中。现在，让我们只关注最上层，即石灰岩，其底部边界由蓝点表示。使用函数 `add_surface_points`，我们可以开始将位置点添加到我们的 `geo_model` 对象中。让我们从一个开始。从图上看，第一个蓝点似乎位于 x 方向约 250 处，深度约 95 米处。由于我们假设剖面位于 y=0，我们可以将 y 保持为 0。我们可以按如下方式输入：

In [ ]:
gp.add_surface_points(
    geo_model=geo_model,
    x=[225],
    y=[0],
    z=[-95],
    elements_names=['Limestone']
)

Now let's plot the data with the section and see how we did:

现在让我们用剖面绘制数据，看看我们做得如何：

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

Great! As you can see, the data point we entered sits right on top of the blue dot on the borehole. Now let's do the same for the other
borehole dots. Conveniently, we can use the same function to add several points at a time:

太棒了！如您所见，我们输入的数据点正好位于钻孔上的蓝点之上。现在让我们对其他钻孔点做同样的事情。方便的是，我们可以使用相同的函数一次添加多个点：

In [ ]:
gp.add_surface_points(
    geo_model=geo_model,
    x=[460, 617],
    y=[0, 0],
    z=[-100, -10],
    elements_names=['Limestone', 'Limestone']
)

p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

## Minimum Data for Model Computation 模型计算的最少数据

Ok, good! Now we have added the position of the bottom of this top layer for each borehole. But is this enough to compute our first layer? 
Well, no. GemPy's approach is based on an implicit interpolation method that requires the following minimum data:

 - **Two surface points** for at least one surface in a structural group/series
 - **One orientation** per structural group/series

Thanks to GemPy's global interpolation approach, once you have one surface defined by two surface points and an orientation in a structural 
group, you can add more surfaces (in the same group) with the minimum of one surface point, as it will now take its orientation information
from the other surface.

So, we are missing an orientation. Let's go ahead and add one. It seems that we can assume a horizontal orientation between the first two 
surface points (apparent, as we have no information about the y-direction). Let's add a corresponding orientation using the function 
`add_orientations`:

好，很好！现在我们已经为每个钻孔添加了该顶层底部的位置。但这足以计算我们的第一层吗？嗯，不。GemPy 的方法基于隐式插值方法，该方法需要以下最少数据：

 - 一个结构组/系列中至少一个表面的 **两个表面点**
 - 每个结构组/系列 **一个方向**

由于 GemPy 的全局插值方法，一旦您在一个结构组中定义了一个由两个表面点和一个方向定义的表面，您就可以添加更多表面（在同一组中），最少只需一个表面点，因为它现在将从另一个表面获取其方向信息。

所以，我们缺少一个方向。让我们继续添加一个。看来我们可以假设前两个表面点之间是水平方向（很明显，因为我们没有关于 y 方向的信息）。让我们使用函数 `add_orientations` 添加相应的方向：

In [ ]:
gp.add_orientations(
    geo_model=geo_model,
    x=[350],
    y=[0],
    z=[-120],
    elements_names=['Limestone'],
    pole_vector=[np.array([0, 0, 1])]
)

p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

Alright, now we actually have sufficient input data to compute the first version of our model. Let's go ahead and do that using `compute_model`:

好了，现在我们实际上有足够的输入数据来计算我们模型的第一个版本。让我们使用 `compute_model` 来做这件事：

In [ ]:
geo_model.update_transform(gp.data.GlobalAnisotropy.NONE)

In [ ]:
gp.compute_model(geo_model, engine_config=gp.data.GemPyEngineConfig())

Now, the model has been computed and is ready to be visualized. Let's see what it looks like in a 2D section:

现在，模型已经计算完毕，可以进行可视化了。让我们看看它在 2D 剖面中的样子：

2D visualization:

2D 可视化：

In [ ]:
gpv.plot_2d(geo_model, cell_number='mid')

We can also plot it together with our cross-section image. By using transparency on the cross-section image, we can overlay it over the 2D model
visualization.

我们还可以将其与我们的横截面图像一起绘制。通过在横截面图像上使用透明度，我们可以将其叠加在 2D 模型可视化之上。

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

You can see how the computed interface runs through the points we defined and is furthermore determined by the one orientation we placed.
Now, let's look at our 3D model:

您可以看到计算出的界面如何穿过我们定义的点，并且还由我们放置的一个方向决定。
现在，让我们看看我们的 3D 模型：

3D visualization:

3D 可视化：

In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False)

Very good, we have successfully computed the first iteration of our model including one lithological interface. You could now go ahead and
fine-tune the model by adding further points or orientations. For now, let's continue.

很好，我们已经成功计算了包含一个岩性界面的模型的第一次迭代。您现在可以继续通过添加更多点或方向来微调模型。现在，让我们继续。

## About Scalar Fields 关于标量场

GemPy's global interpolation approach uses scalar fields to compute models. Each structural group or series has its own scalar field, which
is defined by the data input to all of its elements and at least the minimum data mentioned above. Given that alone, a scalar field will 
already be created "globally", i.e., for the full extent of the modeling space. The defined surface follows one value along that field as
an isosurface. A separate structural element will follow a different isosurface, and once you input additional points and orientations, 
the scalar field will be altered accordingly.

As mentioned, each structural group has its own scalar field. These can be combined to achieve more complex structures and relationships, 
as we will see later.

Let's take a look at the current scalar field for our current group by plotting it in 2D. Keep it in mind as we go ahead and add additional
elements and data.

GemPy 的全局插值方法使用标量场来计算模型。每个结构组或系列都有自己的标量场，该标量场由输入到其所有元素的数据以及至少上述最少数据定义。仅凭这一点，就已经“全局”创建了一个标量场，即针对建模空间的整个范围。定义的表面作为等值面跟随该场沿线的一个值。单独的结构元素将跟随不同的等值面，一旦您输入额外的点和方向，标量场将相应地改变。

如前所述，每个结构组都有自己的标量场。这些可以组合起来以实现更复杂的结构和关系，我们稍后会看到。

让我们通过在 2D 中绘制来查看当前组的当前标量场。在我们继续添加其他元素和数据时，请记住这一点。

In [ ]:
p2d = gpv.plot_2d(
    model=geo_model,
    series_n=0,
    show_data=True,
    show_scalar=True,
    show_lith=False
)

plt.show()

## Adding a Second Lithological Unit 添加第二个岩性单元

To add another unit to our model, we can define it as another structural element and then append it to our `geo_model` object. We do this
for the second unit in the following steps. See how we can already give it a name (let's assume this is a siltstone now), a color corresponding 
to the dot colors in the cross-section, as well as define surface points and orientations. By appending it to `structural_groups[0]`, we are
adding it to our first (and currently only) structural group/series, i.e., it will be in the same stack as our limestone.

要向我们的模型添加另一个单元，我们可以将其定义为另一个结构元素，然后将其附加到我们的 `geo_model` 对象。我们在以下步骤中为第二个单元执行此操作。看看我们如何已经可以给它一个名称（假设这现在是粉砂岩），一个对应于横截面中点颜色的颜色，以及定义表面点和方向。通过将其附加到 `structural_groups[0]`，我们将其添加到我们的第一个（目前也是唯一的）结构组/系列中，即它将与我们的石灰岩处于同一个堆栈中。

In [ ]:
element2 = gp.data.StructuralElement(
    name='Siltstone',
    color='#FFA833',  # color=next(geo_model.structural_frame.color_generator),
    surface_points=gp.data.SurfacePointsTable.from_arrays(
        x=np.array([460]),
        y=np.array([0]),
        z=np.array([-280]),
        names='Siltstone'
    ),
    orientations=gp.data.OrientationsTable.initialize_empty()
)

geo_model.structural_frame.structural_groups[0].append_element(element2)

Now, we can see that this siltstone unit is part of our structural frame. Note below, that it has by default been added below the limestone. 
The order of structural elements within one group only affects the default color assigned. We recommend being consistent in the way you 
choose to order them, and to order them in accordance with geological age. The order of structural groups actually represents geological 
time relationships, with groups at the top being the youngest and lower ones being older. This, together with their `StackRelationType`, 
decides how they affect each other via their individual scalar fields, as we will see later.

现在，我们可以看到这个粉砂岩单元是我们结构框架的一部分。请注意下面，默认情况下它已添加到石灰岩下方。一个组内结构元素的顺序仅影响分配的默认颜色。我们建议在选择排序方式时保持一致，并按照地质年代对其进行排序。结构组的顺序实际上代表了地质时间关系，顶部的组是最年轻的，下面的组是较老的。这连同它们的 `StackRelationType` 决定了它们如何通过各自的标量场相互影响，我们稍后会看到。

In [ ]:
geo_model.structural_frame

In [ ]:
gp.compute_model(geo_model, engine_config=gp.data.GemPyEngineConfig())

In [ ]:
p2d = gpv.plot_2d(geo_model, cell='mid', show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

The 2D visualization of our updated model now shows that the next surface follows the respective point we added. Based on the implicit
modeling approach GemPy uses, given this sparse information we have put in, the siltstone bottom interface otherwise follows the course 
and orientation of the limestone we defined earlier. This is due to the scalar field which was defined by the data input for the structural
group as a whole. In our 2D plot, we can see that this fits quite well with the first borehole, too. Still, let's add the remaining point,
and while we are at it, let's also add the third lithological unit marked in green.

我们更新模型的 2D 可视化现在显示下一个表面跟随我们添加的相应点。基于 GemPy 使用的隐式建模方法，鉴于我们输入的稀疏信息，粉砂岩底部界面在其他方面跟随我们之前定义的石灰岩的走向和方向。这是由于标量场是由整个结构组的数据输入定义的。在我们的 2D 图中，我们可以看到这也非常适合第一个钻孔。尽管如此，让我们添加剩余的点，既然我们正在做，让我们也添加标记为绿色的第三个岩性单元。

First, we add the two missing points to the Siltstone

首先，我们将两个缺失的点添加到粉砂岩中

In [ ]:
gp.add_surface_points(
    geo_model=geo_model,
    x=[225],
    y=[0],
    z=[-270],
    elements_names=['Siltstone']
)

element3 = gp.data.StructuralElement(
    name='Sandstone',
    color='#72A533',  # next(geo_model.structural_frame.color_generator),
    surface_points=gp.data.SurfacePointsTable.from_arrays(
        x=np.array([225, 460]),
        y=np.array([0, 0]),
        z=np.array([-436, -441]),
        names='Sandstone'
    ),
    orientations=gp.data.OrientationsTable.initialize_empty()
)

geo_model.structural_frame.structural_groups[0].append_element(element3)

In [ ]:
gp.compute_model(geo_model)

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

3D visualization:



In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False)

## Discontinuities: Combining Scalar Fields 不连续性：组合标量场

Now that we have created all the lithological units and added all the surface points we got from the boreholes, we have created a very
simple geological model. However, geological scenarios are usually more complex. In GemPy, you can not only combine numerous surface
points and orientations to create elaborate structures but also create various structural groups that affect each other through combinations
of their scalar fields. In the following part, we will look at the right side of our cross-section, where we only have limited data, and
see how we can add a new structural group to create various types of discontinuities, and with that, possibly even meaningful alternative 
model hypotheses.

So, let's define another structural element that will serve to showcase the different types of discontinuities we can implement in GemPy:

既然我们已经创建了所有岩性单元并添加了我们从钻孔获得的所有表面点，我们就创建了一个非常简单的地质模型。然而，地质场景通常更复杂。在 GemPy 中，您不仅可以组合大量表面点和方向来创建复杂的结构，还可以创建各种结构组，这些结构组通过其标量场的组合相互影响。在接下来的部分中，我们将查看横截面的右侧，那里我们只有有限的数据，看看我们如何添加一个新的结构组来创建各种类型的不连续性，并以此可能甚至创建有意义的替代模型假设。

所以，让我们定义另一个结构元素，它将用于展示我们可以在 GemPy 中实现的不同类型的不连续性：

In [ ]:
element_discont = gp.data.StructuralElement(
    name='Discont_Surface',
    color='#990000',  # next(geo_model.structural_frame.color_generator),
    surface_points=gp.data.SurfacePointsTable.from_arrays(
        x=np.array([550, 650]),
        y=np.array([0, 0]),
        z=np.array([-30, -200]),
        names='Discont_Surface'
    ),
    orientations=gp.data.OrientationsTable.from_arrays(
        x=np.array([600]),
        y=np.array([0]),
        z=np.array([-100]),
        G_x=np.array([.3]),
        G_y=np.array([0]),
        G_z=np.array([.3]),
        names='Discont_Surface'
    )
)

To place the discontinuity element in a separate structural group, we need to create one. This is what we do next. Note that we directly 
add the element to the group as we create it, and we define the group's structural relation type as the default 'erode'. We can then insert
the group into the structural frame.

为了将不连续性元素放置在单独的结构组中，我们需要创建一个。这就是我们接下来要做的。请注意，我们在创建组时直接将元素添加到组中，并将组的结构关系类型定义为默认的 'erode'（侵蚀）。然后我们可以将该组插入到结构框架中。

In [ ]:
group_discont = gp.data.StructuralGroup(
    name='Discontinuity',
    elements=[element_discont],
    structural_relation=gp.data.StackRelationType.ERODE,
)

# Insert the fault group into the structural frame:
geo_model.structural_frame.insert_group(0, group_discont)

Let's take a quick look at the state of our structural frame.

让我们快速看一下我们结构框架的状态。

In [ ]:
geo_model.structural_frame

We can now see the two different structural groups: the default one containing the deposition series, and the group containing the
discontinuity. Let's go ahead and compute the model. Once the model has been computed, we can plot the scalar fields of both structural
groups independently.

我们现在可以看到两个不同的结构组：包含沉积系列的默认组，以及包含不连续性的组。让我们继续计算模型。一旦模型计算完毕，我们可以独立绘制两个结构组的标量场。

In [ ]:
gp.compute_model(geo_model, engine_config=gp.data.GemPyEngineConfig())

In [ ]:
p2d = gpv.plot_2d(
    model=geo_model,
    series_n=1,  # This setting will now plot the scalar field used for the deposition
    show_data=True,
    show_scalar=True,
    show_lith=False
)

plt.show()

In [ ]:
p2d = gpv.plot_2d(
    model=geo_model,
    series_n=0,  # This setting will plot the scalar field used for the discontinuity.
    show_data=True,
    show_scalar=True,
    show_lith=False
)

plt.show()

We now have two very different scalar fields. Note how they are each defined by the input data assigned to their respective structural 
elements. Multiple scalar fields in GemPy influence each other depending on (1) their order and (2) their `StackRelationType`. The latter 
defines how a younger (upper) structural group will relate to the older (lower) structural groups and possibly affect their scalar field.

The parameter `StackRelationType` can take the following values:

- `BASEMENT`: Treats all lower groups as the basement.
- `ERODE`: Defines erosive contact/unconformity.
- `ONLAP`: Defines the younger group to be onlapping onto the older groups.
- `FAULT`: Defines the group to be a fault.

We will now take a look at each of these relation types except for the basement type.

我们现在有两个非常不同的标量场。请注意它们是如何分别由分配给其各自结构元素的输入数据定义的。GemPy 中的多个标量场根据 (1) 它们的顺序和 (2) 它们的 `StackRelationType` 相互影响。后者定义了较年轻（上部）的结构组将如何与较老（下部）的结构组相关联，并可能影响它们的标量场。

参数 `StackRelationType` 可以取以下值：

- `BASEMENT`：将所有下部组视为基底。
- `ERODE`：定义侵蚀接触/不整合。
- `ONLAP`：定义较年轻的组超覆在较老的组之上。
- `FAULT`：定义该组为断层。

我们现在将查看除基底类型之外的每种关系类型。

## Erosive Contact 侵蚀接触

For this, we don't have to change anything now, as we already set the `StackRelationType` to be `ERODE`. If we now plot it, we will see 
how this younger structural group erodes all older elements and basically "cuts them out" in our model.

为此，我们现在不需要更改任何内容，因为我们已经将 `StackRelationType` 设置为 `ERODE`。如果我们现在绘制它，我们将看到这个较年轻的结构组如何侵蚀所有较老的元素，并基本上在我们的模型中“切除它们”。

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

3D visualization:

3D 可视化：

In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False, show_lith=False)

We can see how all units of the depositional series stop at the contact with the new discontinuity group. However, this doesn't look 
quite right, and it in particular doesn't fit the surface point that was observed in the third borehole. So let's try another relation type.

我们可以看到沉积系列的所有单元如何在与新的不连续性组接触处停止。然而，这看起来不太对劲，特别是它不符合在第三个钻孔中观察到的表面点。所以让我们尝试另一种关系类型。

## Onlapping 超覆

Let's change the relation type from `ERODE` to `ONLAP` to achieve a different type of discontinuity and then plot it.

让我们将关系类型从 `ERODE` 更改为 `ONLAP` 以实现不同类型的不连续性，然后绘制它。

In [ ]:
geo_model.structural_frame.structural_groups[0].structural_relation = gp.data.StackRelationType.ONLAP

In [ ]:
gp.compute_model(geo_model, engine_config=gp.data.GemPyEngineConfig())

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

3D visualization:

3D 可视化：

In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False, show_lith=False)

Now the unit defined as part of the discontinuity group is onlapping onto the uppermost surface of the default group and ends there. 
This also doesn't really make sense considering the data given, so let's try the last relation type.

现在，定义为不连续性组一部分的单元超覆在默认组的最上层表面上并在那里结束。考虑到给定的数据，这也没有真正的意义，所以让我们尝试最后一种关系类型。

## Faults 断层

Let's change the relation type to `FAULT` and plot the results. For a fault, we also need to make use of the function `set_is_fault`.

让我们将关系类型更改为 `FAULT` 并绘制结果。对于断层，我们还需要使用函数 `set_is_fault`。

In [ ]:
geo_model.structural_frame.structural_groups[0].structural_relation = gp.data.StackRelationType.FAULT

In [ ]:
gp.set_is_fault(
    frame=geo_model.structural_frame,
    fault_groups=['Discontinuity']
)

See that the fault relations field in the structural frame now indicates that the fault affects the default formation, i.e., offsets it.
Let's compute our model including the fault and see what that looks like.

看到结构框架中的断层关系字段现在表明断层影响默认地层，即使其发生偏移。让我们计算包含断层的模型，看看它是什么样子的。

In [ ]:
gp.compute_model(geo_model, engine_config=gp.data.GemPyEngineConfig())

In [ ]:
p2d = gpv.plot_2d(geo_model, show=False)
p2d.axes[0].imshow(img, origin='upper', alpha=.8, extent=(0, 780, -582, 0))

# Enable grid with minor ticks
p2d.axes[0].grid(which='both')  # Enable both major and minor grids
p2d.axes[0].minorticks_on()  # Enable minor ticks

# Customize the appearance of the grid if needed
p2d.axes[0].grid(which='major', linestyle='--', linewidth='0.8', color='gray')
p2d.axes[0].grid(which='minor', linestyle=':', linewidth='0.4', color='gray')

plt.show()

3D visualization:

3D 可视化：

In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False, show_lith=False)

In the 2D and 3D visualizations, we can now see how the insertion of a fault has created a viable alternative hypothesis that fits the 
original data in the cross-section. Instead of a larger syncline structure and upward-bending of the whole depositional series, we can 
now explain the shallower depth of the limestone in the third borehole by the effects of a reverse fault. In this model realization,
all lithological units are oriented near-horizontal but are offset by the fault. Note that in GemPy, the degree of offset is defined 
by the location of the surface points on each side. If there is no data on one side of a fault, a very large offset will be assumed.

Note also that by implementing a fault, the scalar field of the depositional series has been affected by the offset.

在 2D 和 3D 可视化中，我们现在可以看到断层的插入如何创建了一个符合横截面原始数据的可行替代假设。我们现在可以通过逆断层的影响来解释第三个钻孔中石灰岩较浅的深度，而不是较大的向斜结构和整个沉积系列的向上弯曲。在这个模型实现中，所有岩性单元都接近水平，但被断层偏移。请注意，在 GemPy 中，偏移程度由每侧表面点的位置定义。如果断层一侧没有数据，则假设偏移量非常大。

还要注意，通过实施断层，沉积系列的标量场已受到偏移的影响。

In [ ]:
p2d = gpv.plot_2d(
    model=geo_model,
    series_n=1,  # This will plot the scalar field used for the fault
    show_data=True,
    show_scalar=True,
    show_lith=False
)

plt.show()

## Topography and Geological Maps 地形和地质图（未读）

In GemPy, we can add more grid types for different purposes, such as to add topography to our model. In this following section,
we will exemplify this by creating a random topography grid which allows us to intersect the surfaces as well as compute a high-resolution
geological map. GemPy has a built-in function to generate random topography. After executing it, a topography grid will be added to the 
`geo_model`. It can be directly visualized in 2D and 3D.

在 GemPy 中，我们可以为不同的目的添加更多网格类型，例如向我们的模型添加地形。在接下来的部分中，我们将通过创建一个随机地形网格来举例说明这一点，该网格允许我们与表面相交并计算高分辨率地质图。GemPy 有一个内置函数来生成随机地形。执行后，地形网格将被添加到 `geo_model` 中。它可以直接在 2D 和 3D 中可视化。

GemPy offers built-in tools to manage topographic data through gdal.
For demonstration, we'll create a random topography:

GemPy 提供了内置工具来通过 gdal 管理地形数据。
为了演示，我们将创建一个随机地形：

In [ ]:
gp.set_topography_from_random(
    grid=geo_model.grid,
    fractal_dimension=1.9,
    d_z=np.array([-150, 0]),
    topography_resolution=np.array([200, 200])
)

In [ ]:
gpv.plot_2d(geo_model, show_topography=True)

In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False, show_topography=True, show_lith=False)

If we now also re-compute our geological model, the generated topography grid will display the lithological units that intersect it, 
i.e., which outcrop at the surface. We can therefore display a geological map based on our topography and the underlying 3D geological model. 
To plot a top-down view of this map, you can pass the arguments `section_names=['topography']` and `show_topography=True` in the plotting
function as shown below.

如果我们现在也重新计算我们的地质模型，生成的地形网格将显示与其相交的岩性单元，即在表面露头的单元。因此，我们可以基于我们的地形和底层的 3D 地质模型显示地质图。要绘制此地图的自上而下的视图，您可以在绘图函数中传递参数 `section_names=['topography']` 和 `show_topography=True`，如下所示。

In [ ]:
gp.compute_model(geo_model)

In [ ]:
gpv.plot_2d(geo_model, section_names=['topography'], show_topography=True)

In [ ]:
gpv.plot_3d(geo_model, show_surfaces=True, image=False, show_topography=True)

We can now see how the topography displays the color of the lithologies outcropping at the surface, together with topographical contour lines.

While this topography is random, GemPy also has the capability to include real topography files and arrays via the functions 
`set_topography_from_file` and `set_topography_from_arrays`.

我们现在可以看到地形如何显示在表面露头的岩性的颜色，以及地形等高线。

虽然这个地形是随机的，但 GemPy 也有能力通过函数 `set_topography_from_file` 和 `set_topography_from_arrays` 包含真实的地形文件和数组。

## Extracting Model Solutions 提取模型解

Once you have built a model, you might not only want to visualize it, but also further analyze it or export it for further utilization. 
For this, it is good to know that the solutions from modeling are stored in `geo_model.solutions` and can be returned from there. This
includes the following outputs in particular:
- `geo_model.solutions.dc_meshes`: A list of the surface meshes in the model with the location of vertices and edges for each.
- `geo_model.solutions.raw_arrays`: An object containing numerous arrays that define various parts of the model. Of particular importance
are the lithology block (`lith_block`), the fault block (`fault_block`), and the scalar field matrix (`scalar_field_matrix`).

一旦您构建了一个模型，您可能不仅想将其可视化，还想进一步分析它或将其导出以供进一步利用。为此，很高兴知道建模的解存储在 `geo_model.solutions` 中，并且可以从那里返回。这特别包括以下输出：
- `geo_model.solutions.dc_meshes`：模型中表面网格的列表，包含每个网格的顶点和边缘的位置。
- `geo_model.solutions.raw_arrays`：一个包含定义模型各个部分的众多数组的对象。特别重要的是岩性块 (`lith_block`)、断层块 (`fault_block`) 和标量场矩阵 (`scalar_field_matrix`)。

## Mesh Solutions 网格解

Let's take a quick look at how we can return some key information from `geo_model.solutions`. Starting with meshes, we can see that the list 
`dc_meshes` can be indexed to return specific meshes and their respective vertices or edges. Please note that the order will be the same as 
in our `structural_frame`, i.e., the index `[0]` will return the first and top surface, in our case, the discontinuity surface.

让我们快速看一下如何从 `geo_model.solutions` 返回一些关键信息。从网格开始，我们可以看到列表 `dc_meshes` 可以被索引以返回特定的网格及其各自的顶点或边缘。请注意，顺序将与我们的 `structural_frame` 中的顺序相同，即索引 `[0]` 将返回第一个和顶部的表面，在我们的例子中是不连续性表面。

In [ ]:
vertices_0 = geo_model.solutions.dc_meshes[0].vertices
edges_0 = geo_model.solutions.dc_meshes[0].edges
print(type(vertices_0), vertices_0, edges_0)

> out:
<class 'numpy.ndarray'> [[ 0.45129359 -0.23915722 -0.63081408]
 [ 0.43619337 -0.23915722 -0.58985021]
 [ 0.45123358 -0.20726947 -0.63101987]
 ...
 [ 0.21781076  0.2072733  -0.04150489]
 [ 0.23731025  0.23916798 -0.07939452]
 [ 0.21703678  0.23916084 -0.04243479]] [[  3   1   2]
 [  8   6   3]
 [  9   7   8]
 ...
 [316 314 313]
 [318 323 314]
 [327 325 323]]

We can see that the vertices for this mesh were returned as a Numpy array with values for *x*, *y*, and *z* positions for each vertex. 
However, the values don't correspond with our model extent. That is because they have been transformed in GemPy. To return the values
corresponding to the original coordinate system, we can invert this transformation as follows:

我们可以看到此网格的顶点作为 Numpy 数组返回，其中包含每个顶点的 *x*、*y* 和 *z* 位置的值。然而，这些值与我们的模型范围不对应。这是因为它们在 GemPy 中已被转换。要返回对应于原始坐标系的值，我们可以按如下方式反转此转换：

In [ ]:
geo_model.input_transform.apply_inverse(vertices_0)

>out:
array([[ 774.81417593, -187.49925971, -559.55823517],
       [ 762.97560303, -187.49926005, -527.44256826],
       [ 774.76712736, -162.49926701, -559.71957915],
       ...,
       [ 591.76363561,  162.50226762,  -97.53983597],
       [ 607.05123778,  187.5076957 , -127.24530088],
       [ 591.15683725,  187.50209584,  -98.26887806]], shape=(338, 3))

## Lithology Block 岩性块

The lithology block is an array that, for a given model realization/solution, returns the ID of the lithology for each voxel. Note below
that the `lith_block` first returns all values in the shape of one row. You might need to reshape it as shown below. For a regular grid,
you can reshape it using the resolution used in `geo_model`.

岩性块是一个数组，对于给定的模型实现/解，它返回每个体素的岩性 ID。请注意下面，`lith_block` 首先以一行的形状返回所有值。您可能需要如下所示对其进行重塑。对于规则网格，您可以使用 `geo_model` 中使用的分辨率对其进行重塑。

In [ ]:
lith_block = geo_model.solutions.raw_arrays.lith_block
print(lith_block.shape, lith_block)

> out:
(125000,) [5 5 5 ... 3 3 2]

In [ ]:
lith_block = lith_block.reshape(50, 50, 50)
print(lith_block.shape, lith_block)

> out:
(50, 50, 50) [[[5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  ...
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]]

 [[5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  ...
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]]

 [[5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  ...
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]
  [5 5 5 ... 2 2 2]]

 ...

 [[5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  ...
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]]

 [[5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  ...
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]]

 [[5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  ...
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]
  [5 5 5 ... 3 3 2]]]

## Grid Values 网格值

Apart from these solutions, you might also need to return grid values. You can access the values for each grid in your `geo_model` 
object via `geo_model.grid` as shown below.

除了这些解之外，您可能还需要返回网格值。您可以如下所示通过 `geo_model.grid` 访问 `geo_model` 对象中每个网格的值。

In [ ]:
geo_model.grid.regular_grid.values

> out:
array([[   7.8 , -196.  , -576.18],
       [   7.8 , -196.  , -564.54],
       [   7.8 , -196.  , -552.9 ],
       ...,
       [ 772.2 ,  196.  ,  -29.1 ],
       [ 772.2 ,  196.  ,  -17.46],
       [ 772.2 ,  196.  ,   -5.82]], shape=(125000, 3))

In [ ]:
geo_model.grid.topography.values

> out:
array([[   0.        , -200.        , -103.92301797],
       [   0.        , -197.98994975, -104.04483853],
       [   0.        , -195.9798995 , -105.20968466],
       ...,
       [ 780.        ,  195.9798995 , -106.1938437 ],
       [ 780.        ,  197.98994975, -104.64212323],
       [ 780.        ,  200.        , -103.63462951]], shape=(40000, 3))